## Block Matching Regisrtation

This notebook outputs transformaed pointclouds + alignmnet matrices + warped images. 
This notebut is run in WSL since it is using ants , which is not awailable for windows. 

For vizualisation, go to " NC_Control_View" notebook , which is run from windows ... 

In [38]:
import sys
# win:
# project_path = 'D:/Code/repos/gad1b-redistribution/src'
# wsl:
project_path = '/mnt/d/Code/repos/pwreg'
sys.path.insert(1, f'{project_path}/pwreg')

from core.core import *
from visualize.viewers import *

import inspect
# print(inspect.getsource(Something))

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
project_path

'/mnt/d/Code/repos/pwreg'

### Whole image alignment ( TODO : allow rigid ) 

In [4]:
fixed = Image([0.658, 0.23, 0.23], filename = f'{project_path}/data/raw/img/no_change_control/Image_1-20FM.ome.tif', mask= {'zmin': 51,'zmax': 164 + 1}) # tp 2
moving = Image([0.658, 0.23, 0.23], filename = f'{project_path}/data/raw/img/no_change_control/Image_1-20FJ.ome.tif', mask= {'zmin': 51,'zmax': 168 + 1}) # tp 1

print(f'shape\nFixed {fixed.shape},\nMoving {moving.shape}')
print(f'masks :\nFixed {fixed.mask},\nMoving {moving.mask}')

shape
Fixed (114, 1024, 1024),
Moving (118, 1024, 1024)
masks :
Fixed {'zmin': 51, 'zmax': 165, 'xmin': None, 'xmax': None, 'ymin': None, 'ymax': None},
Moving {'zmin': 51, 'zmax': 169, 'xmin': None, 'xmax': None, 'ymin': None, 'ymax': None}


In [5]:
# put the whole image size here
voxels_f = fixed.split([114,1024,1024],0)
voxels_m = moving.split([118,1024,1024],0)

In [6]:
pair = BlockPair(voxels_f[0],voxels_m[0])
pair.register()

In [20]:
save_image = 0
if save_image: 
    tif.imsave(f'{project_path}/data/processed/registered/no_change_control/1-20FJ_to_1-20FM_affine.tif', pair.warp(), imagej=True)
    

In [33]:
af = {'matrix': pair.alignment['affine'].matrix, 'center' : pair.alignment['affine'].center}
with open(f'{project_path}/data/processed/registered/no_change_control/1-20FJ_to_1-20FM_affine.pkl', 'wb') as f:
    pickle.dump(af, f)

In [34]:
# with open(f'{project_path}/data/processed/registered/no_change_control/1-20FJ_to_1-20FM_affine.pkl', 'rb') as f:
#     af = pickle.load(f)

{'matrix': array([[ 1.01343539e+00,  5.01786922e-03, -3.49930976e-02,
          0.00000000e+00],
        [-7.74445434e-04,  1.00188341e+00, -1.23891263e-02,
          0.00000000e+00],
        [ 3.71186197e-04,  1.20298269e-02,  1.00269345e+00,
          0.00000000e+00],
        [-1.54117430e-01, -2.22441956e+00,  3.30081141e+00,
          1.00000000e+00]]),
 'center': array([0., 0., 0.])}

### Block affine registration

In [14]:
# use the pre-registered 
prereg = Image([0.658, 0.23, 0.23],img = pair.warp())

Let's try with the overlap:

In [15]:
voxels_f = fixed.split([18,102,102],[2,5,5])
voxels_m = prereg.split([18,102,102],[2,5,5])

/mnt/d/Code/repos/pwreg/pwreg/core/core.py:89: UserWarning:

Specified voxel size + overlap don't cover the whole image.Image size is [ 114 1024 1024], block size [ 18 102 102], overlap [2 5 5] results in [ 7.         10.50515464 10.50515464] blocks.
Leaving some image out. 



In [16]:
pairs = [BlockPair(voxf,voxm) for voxf,voxm in zip(voxels_f,voxels_m)]
for pair in pairs: 
    pair.register()

In [26]:
BlockPairView(pairs, 10).write_volume(f'{project_path}/data/processed/registered/no_change_control/pw_registration_moving_1-20FJ_overlap_NN.tif')
BlockView(voxels_f,10).write_volume(f'{project_path}/data/processed/registered/no_change_control/pw_registration_fixed_1-20FM_overlap_NN.tif')

(1, 1, 186, 1110, 1110)


### Load points

In [29]:
import pandas as pd

segmentation_folder = '/mnt/d/Code/repos/psd95_segmentation/models/two_small_cnns/predictions/Gad1b_2021/Sequential_2tp_control/'
ml_seg1 = pd.read_csv(f'{segmentation_folder}ML_segmentation_1-20GA_12-22-2021_2CNN_gui.csv')
ml_seg2 = pd.read_csv(f'{segmentation_folder}ML_segmentation_1-20G6_12-22-2021_2CNN_gui.csv')

In [40]:
prob_thr = 0.5
ptc1 = Points(ml_seg1[['X','Y','Z']][ml_seg1["prob"]>prob_thr].to_numpy(), units='pix', resolution=[0.23, 0.23, 0.658])
ptc2 = Points(ml_seg2[['X','Y','Z']][ml_seg2["prob"]>prob_thr].to_numpy(), units='pix', resolution=[0.23, 0.23, 0.658])

In [41]:
ptc1 = ptc1.crop({'zmin': 51,'zmax': 164 + 1}, units='pix')
ptc1 = ptc1.recenter([0,0,51], units = 'pix')

pyc2 = ptc2.crop({'zmin': 51,'zmax': 168 + 1}, units='pix')
ptc2 = ptc2.recenter([0,0,51], units = 'pix')

In [42]:
ptc2

Number of points : 12961
Resolution : [0.23  0.23  0.658]
Coordinates :
- pixels
[[ 231  178   25]
 [ 181  160   27]
 [ 162  187   27]
 ...
 [ 822  993  115]
 [ 858 1010  115]
 [ 781  870  116]]
- physical units
[[ 53.13   40.94   16.45 ]
 [ 41.63   36.8    17.766]
 [ 37.26   43.01   17.766]
 ...
 [189.06  228.39   75.67 ]
 [197.34  232.3    75.67 ]
 [179.63  200.1    76.328]]

In [45]:
# apply the original global transform
with open(f'{project_path}/data/processed/registered/no_change_control/1-20FJ_to_1-20FM_affine.pkl', 'rb') as f:
    af = pickle.load(f)
ptc2 = ptc2.transform(AffineTransform(af['matrix'],center = af['center']), units='phs')
ptc2

Number of points : 12961
Resolution : [0.23  0.23  0.658]
Coordinates :
- pixels
[[ 236.  163.   28.]
 [ 184.  145.   31.]
 [ 165.  172.   32.]
 ...
 [ 842.  993.   97.]
 [ 879. 1011.   96.]
 [ 800.  870.  100.]]
- physical units
[[ 54.20705265  37.58563906  18.41224399]
 [ 42.40361     33.35094635  20.64988074]
 [ 37.90574876  39.54105817  20.80440172]
 ...
 [193.56218151 228.45929742  63.69346558]
 [202.05990594 232.46368844  63.01179321]
 [183.92201709 199.99143649  65.72260089]]

In [51]:
ptc2.to_json(f'{project_path}/data/processed/registered/no_change_control/ptc2_global_tansformed.json')
ptc1.to_json(f'{project_path}/data/processed/registered/no_change_control/ptc1_global.json')

Now need to look at the image and pointcloud ... 

In [115]:
class ImagePointsView:
    def __init__(self, img, ptc):
        self.img = img
        self.ptc = ptc

    def view_in_napari(self):
        """
        Display image with the corresponding point cloud.
        """
        resolution = self.img.resolution
        with napari.gui_qt():
            viewer = napari.Viewer()
            viewer.add_image(self.img.img,
                             scale=resolution,
                             name='image',
                             colormap='grey',
                             blending='additive')

            viewer.add_layer(FixedPoints(
                            self.ptc.xyz['pix'],
                            ndim=3,
                            size=2,
                            edge_width=1,
                            scale=resolution,
                            name='points',
                            face_color='cyan'))

In [116]:
imgptc = ImagePointsView(fixed,ptc1)
imgptc.view_in_napari()

AttributeError: module 'scipy' has no attribute 'sparse'